In [1]:
!pip install -r requirements.txt

In [ ]:

import os
import whisper
from pyannote.audio import Pipeline
from pathlib import Path
import torch
import torchaudio

audio_file_path = "/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/data_samples/RES0029.mp3"  # Your audio file
output_file_path = "/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/data_samples/transcription1.txt"  # Where to save the result


whisper_model = whisper.load_model("base")

HF_TOKEN = os.getenv("HF_TOKEN")
diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=HF_TOKEN)


/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()
/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/speechbrain/utils/to

In [3]:


diarization = diarization_pipeline(audio_file_path)
segments = []
for turn, _, speaker in diarization.itertracks(yield_label=True):
    segments.append({
        "start": turn.start,
        "end": turn.end,
        "speaker": speaker
    })

print(f"Found {len(segments)} speaker segments.")



/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequence

Found 150 speaker segments.


In [4]:
def transcribe_segment(audio_path, start_time, end_time, model):
    waveform, sample_rate = torchaudio.load(audio_path)
    # Trim the audio to the segment
    start_sample = int(start_time * sample_rate)
    end_sample = int(end_time * sample_rate)
    segment_waveform = waveform[:, start_sample:end_sample]

    # Save temporary segment file
    temp_path = "temp_segment.wav"
    torchaudio.save(temp_path, segment_waveform, sample_rate)

    # Transcribe with Whisper
    result = model.transcribe(temp_path)
    return result["text"]


In [5]:
speaker_map = {
    "SPEAKER_00": "Doctor",
    "SPEAKER_01": "Patient"
}
speaker_transcripts = []
for seg in segments:
    text = transcribe_segment(audio_file_path, seg["start"], seg["end"], whisper_model)
    speaker_name = speaker_map.get(seg["speaker"], seg["speaker"])
    speaker_transcripts.append(f'{seg["speaker"]}: {text}')

with open(output_file_path, "w", encoding="utf-8") as f:
    for line in speaker_transcripts:
        f.write(line + "\n")

print(f"Transcription completed and saved to {output_file_path}")

/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/torchaudio/_backend/utils.py:337: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.save_with_torchcodec` under the hood. Some parameters like format, encoding, bits_per_sample, buffer_size, and ``backend`` will be ignored. We recommend that you port your code to rely directly on To

Transcription completed and saved to /Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/data_samples/transcription1.txt


/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/torchaudio/_backend/utils.py:337: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.save_with_torchcodec` under the hood. Some parameters like format, encoding, bits_per_sample, buffer_size, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's encoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.encoders.AudioEncoder
  warnings.warn(
/Users/anushkapawar/Documents/3MORE Work/Medical_call_analysis/.venv/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
